# Deep Learning for Drug Discovery with DeepChem

This notebook demonstrates how to use DeepChem for molecular property prediction and drug discovery tasks.

## Learning Objectives
- Load and preprocess molecular datasets
- Create molecular featurizations
- Build and train deep learning models
- Evaluate model performance for drug discovery

In [ ]:
# Import required libraries
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from sklearn.metrics import r2_score, mean_absolute_error
import warnings
import logging
import os
import wandb
from datetime import datetime

# Suppress various warnings
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Suppress DeepChem normalization warnings
import sys
from contextlib import redirect_stdout, redirect_stderr
from io import StringIO

# Enhanced Weights & Biases Setup
print("🚀 Setting up Weights & Biases tracking...")

# Login to wandb with your API key
wandb.login(key="b4f102d87161194b68baa7395d5862aa3f93b2b7", relogin=True)

# Initialize experiment tracking
experiment_config = {
    "framework": "deepchem",
    "task": "molecular_property_prediction",
    "model_type": "deep_learning",
    "dataset": "drug_discovery_demo",
    "notebook": "03_deepchem_drug_discovery",
    "timestamp": datetime.now().isoformat(),
    "environment": "jupyter",
    "libraries": {
        "deepchem": dc.__version__,
        "numpy": np.__version__,
        "pandas": pd.__version__
    }
}

# Start wandb experiment
run = wandb.init(
    project="chemml-experiments",
    name=f"deepchem_drug_discovery_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    config=experiment_config,
    tags=["deepchem", "drug_discovery", "molecular_ml", "tutorial"],
    notes="Deep learning for drug discovery using DeepChem - comprehensive tutorial with wandb tracking"
)

print(f"✅ DeepChem version: {dc.__version__}")
print(f"✅ Weights & Biases experiment started: {run.name}")
print(f"📊 View experiment at: {run.url}")
print("🧪 Ready for molecular drug discovery experiments!")

# Helper function to log molecular data to wandb
def log_molecular_metrics(y_true, y_pred, dataset_name="", step=None):
    """Log comprehensive molecular prediction metrics to wandb."""
    try:
        from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
        
        # Calculate metrics
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true, y_pred)
        
        # Create metrics dictionary
        metrics = {
            f"{dataset_name}mse": mse,
            f"{dataset_name}mae": mae,
            f"{dataset_name}rmse": rmse,
            f"{dataset_name}r2_score": r2,
        }
        
        # Log to wandb
        if step is not None:
            wandb.log(metrics, step=step)
        else:
            wandb.log(metrics)
        
        # Create and log prediction plot
        plt.figure(figsize=(8, 6))
        plt.scatter(y_true, y_pred, alpha=0.6, s=20)
        plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'{dataset_name}Predictions vs Actual (R² = {r2:.3f})')
        plt.grid(True, alpha=0.3)
        
        # Log plot to wandb
        wandb.log({f"{dataset_name}prediction_plot": wandb.Image(plt)}, step=step)
        plt.show()
        
        print(f"📈 Logged metrics for {dataset_name}: R² = {r2:.3f}, RMSE = {rmse:.3f}")
        
    except Exception as e:
        print(f"⚠️ Error logging metrics: {e}")

print("🎯 Helper functions loaded and ready!")

## 1. Loading and Exploring Molecular Data

In [ ]:
# Create a sample dataset with SMILES and properties
# In practice, you would load this from a file or database
sample_data = {
    'smiles': [
        'CCO',  # Ethanol
        'CC(=O)O',  # Acetic acid
        'c1ccccc1',  # Benzene
        'CC(=O)Nc1ccc(O)cc1',  # Paracetamol
        'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O',  # Ibuprofen
        'CN1C=NC2=C1C(=O)N(C(=O)N2C)C',  # Caffeine
        'CC(C)NCC(C1=CC(=C(C=C1)O)CO)O',  # Salbutamol
        'CC1=CC=C(C=C1)C2=CC=C(C=C2)C',  # 4,4'-Dimethylbiphenyl
        'C1=CC=C(C=C1)C(=O)O',  # Benzoic acid
        'CC(C)(C)C1=CC=C(C=C1)O'  # 4-tert-Butylphenol
    ],
    'logp': [−0.31, −0.17, 2.13, 0.46, 3.97, −0.07, 0.1, 4.79, 1.87, 3.31],  # LogP values
    'solubility': [0.0, 0.0, -2.13, -1.46, -3.97, -0.8, -1.5, -4.79, -1.87, -3.31],  # Log solubility
    'mw': [46.07, 60.05, 78.11, 151.16, 206.28, 194.19, 239.31, 182.26, 122.12, 150.22]  # Molecular weight
}

df = pd.DataFrame(sample_data)
print("Sample molecular dataset:")
print(df.head())
print(f"\nDataset shape: {df.shape}")

## 2. Molecular Featurization

In [ ]:
# Different featurization methods in DeepChem

# 1. Circular Fingerprints (ECFP)
ecfp_featurizer = dc.feat.CircularFingerprint(size=1024, radius=2)
ecfp_features = ecfp_featurizer.featurize(df['smiles'])

print(f"ECFP features shape: {ecfp_features.shape}")
print(f"ECFP features for first molecule: {ecfp_features[0][:10]}...")  # First 10 features

# 2. RDKit Descriptors
rdkit_featurizer = dc.feat.RDKitDescriptors()
rdkit_features = rdkit_featurizer.featurize(df['smiles'])

print(f"\nRDKit descriptors shape: {rdkit_features.shape}")
print(f"RDKit descriptors for first molecule: {rdkit_features[0][:5]}...")  # First 5 descriptors

# 3. Coulomb Matrix (for small molecules)
try:
    coulomb_featurizer = dc.feat.CoulombMatrix(max_atoms=50)
    coulomb_features = coulomb_featurizer.featurize(df['smiles'])
    print(f"\nCoulomb Matrix features shape: {coulomb_features.shape}")
except Exception as e:
    print(f"\nCoulomb Matrix featurization failed: {e}")

## 3. Creating DeepChem Datasets

In [ ]:
# Create DeepChem datasets for different properties

# Dataset for LogP prediction
logp_dataset = dc.data.NumpyDataset(
    X=ecfp_features,
    y=np.array(df['logp']).reshape(-1, 1),
    ids=df['smiles']
)

# Dataset for solubility prediction
solubility_dataset = dc.data.NumpyDataset(
    X=ecfp_features,
    y=np.array(df['solubility']).reshape(-1, 1),
    ids=df['smiles']
)

print(f"LogP dataset: {logp_dataset}")
print(f"Features shape: {logp_dataset.X.shape}")
print(f"Labels shape: {logp_dataset.y.shape}")
print(f"Number of tasks: {logp_dataset.n_tasks}")
print(f"Number of features: {logp_dataset.n_features}")

## 4. Data Splitting and Preprocessing

In [ ]:
# Split the dataset into train/validation/test
# For small datasets, we'll use a simple random split
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
    logp_dataset, 
    train_dir=None,
    valid_dir=None,
    test_dir=None,
    frac_train=0.6,
    frac_valid=0.2,
    frac_test=0.2
)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(valid_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Apply normalization
normalizer = dc.trans.NormalizationTransformer(
    transform_y=True, dataset=train_dataset
)

train_dataset_norm = normalizer.transform(train_dataset)
valid_dataset_norm = normalizer.transform(valid_dataset)
test_dataset_norm = normalizer.transform(test_dataset)

print("\nDatasets normalized successfully!")

## 5. Building and Training Deep Learning Models

In [ ]:
# Create different types of models

# 1. Multi-layer Perceptron (MLP)
mlp_model = dc.models.MultitaskRegressor(
    n_tasks=1,
    n_features=1024,  # ECFP size
    layer_sizes=[512, 256, 128],
    dropouts=0.2,
    learning_rate=0.001,
    batch_size=32
)

print("Training MLP model...")
mlp_model.fit(train_dataset_norm, nb_epoch=50)
print("MLP training completed!")

# 2. Random Forest (for comparison)
rf_model = dc.models.SklearnModel(
    model_instance=dc.models.sklearn_models.RandomForestRegressor(
        n_estimators=100, random_state=42
    )
)

print("\nTraining Random Forest model...")
rf_model.fit(train_dataset)
print("Random Forest training completed!")

## 6. Model Evaluation

In [ ]:
# Evaluate models on test set
metric = dc.metrics.Metric(dc.metrics.r2_score)

# MLP evaluation
mlp_train_score = mlp_model.evaluate(train_dataset_norm, [metric])
mlp_valid_score = mlp_model.evaluate(valid_dataset_norm, [metric])
mlp_test_score = mlp_model.evaluate(test_dataset_norm, [metric])

# Random Forest evaluation
rf_train_score = rf_model.evaluate(train_dataset, [metric])
rf_valid_score = rf_model.evaluate(valid_dataset, [metric])
rf_test_score = rf_model.evaluate(test_dataset, [metric])

print("Model Performance (R² Score):")
print("\nMLP Model:")
print(f"  Train R²: {mlp_train_score['r2_score']:.3f}")
print(f"  Valid R²: {mlp_valid_score['r2_score']:.3f}")
print(f"  Test R²:  {mlp_test_score['r2_score']:.3f}")

print("\nRandom Forest Model:")
print(f"  Train R²: {rf_train_score['r2_score']:.3f}")
print(f"  Valid R²: {rf_valid_score['r2_score']:.3f}")
print(f"  Test R²:  {rf_test_score['r2_score']:.3f}")

## 7. Making Predictions and Visualization

In [ ]:
# Make predictions on test set
mlp_predictions = mlp_model.predict(test_dataset_norm)
rf_predictions = rf_model.predict(test_dataset)

# Get actual values
actual_values = test_dataset.y.flatten()

# Denormalize MLP predictions
mlp_predictions_denorm = normalizer.untransform(test_dataset_norm)
mlp_pred_values = mlp_predictions_denorm.y.flatten()
rf_pred_values = rf_predictions.flatten()

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# MLP predictions
ax1.scatter(actual_values, mlp_pred_values, alpha=0.7, color='blue', s=100)
ax1.plot([min(actual_values), max(actual_values)], 
         [min(actual_values), max(actual_values)], 'r--', lw=2)
ax1.set_xlabel('Actual LogP')
ax1.set_ylabel('Predicted LogP')
ax1.set_title('MLP Model Predictions')
ax1.grid(True, alpha=0.3)

# Random Forest predictions
ax2.scatter(actual_values, rf_pred_values, alpha=0.7, color='green', s=100)
ax2.plot([min(actual_values), max(actual_values)], 
         [min(actual_values), max(actual_values)], 'r--', lw=2)
ax2.set_xlabel('Actual LogP')
ax2.set_ylabel('Predicted LogP')
ax2.set_title('Random Forest Predictions')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print prediction details
print("Test Set Predictions:")
for i, smiles in enumerate(test_dataset.ids):
    print(f"{smiles}: Actual={actual_values[i]:.2f}, "
          f"MLP={mlp_pred_values[i]:.2f}, RF={rf_pred_values[i]:.2f}")

## 8. Feature Importance Analysis

In [ ]:
# For Random Forest, we can get feature importance
if hasattr(rf_model.model_instance, 'feature_importances_'):
    feature_importance = rf_model.model_instance.feature_importances_
    
    # Get top 20 most important features
    top_features_idx = np.argsort(feature_importance)[-20:]
    top_features_importance = feature_importance[top_features_idx]
    
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(top_features_importance)), top_features_importance)
    plt.xlabel('Feature Importance')
    plt.ylabel('ECFP Bit Index')
    plt.title('Top 20 Most Important ECFP Features (Random Forest)')
    plt.yticks(range(len(top_features_importance)), top_features_idx)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"Most important feature index: {top_features_idx[-1]}")
    print(f"Highest importance value: {top_features_importance[-1]:.4f}")

## 9. Virtual Screening Example

In [ ]:
# Example of virtual screening with new molecules
new_molecules = [
    'CC(C)C1=CC=C(C=C1)C(=O)O',  # Similar to ibuprofen
    'CC1=CC=CC=C1C(=O)O',  # Toluic acid
    'C1=CC=C(C=C1)CCO',  # Phenethyl alcohol
    'CC(C)(C)C1=CC=CC=C1'  # tert-Butylbenzene
]

print("Virtual Screening Results:")
print("Predicting LogP for new molecules...\n")

# Featurize new molecules
new_features = ecfp_featurizer.featurize(new_molecules)
new_dataset = dc.data.NumpyDataset(X=new_features, ids=new_molecules)

# Make predictions
rf_new_predictions = rf_model.predict(new_dataset)

# Display results
for i, smiles in enumerate(new_molecules):
    predicted_logp = rf_new_predictions[i, 0]
    
    # Classify based on LogP
    if predicted_logp < 1:
        classification = "Hydrophilic"
    elif predicted_logp < 3:
        classification = "Moderate"
    else:
        classification = "Lipophilic"
    
    print(f"Molecule: {smiles}")
    print(f"  Predicted LogP: {predicted_logp:.2f}")
    print(f"  Classification: {classification}")
    print()

## Exercise

Try the following:
1. Load a real dataset from DeepChem (e.g., Tox21, BACE)
2. Try different featurization methods (GraphConv, MPNN)
3. Implement cross-validation for more robust evaluation
4. Explore multi-task learning for predicting multiple properties
5. Use graph neural networks for molecular property prediction

## Next Steps
- Explore DeepChem's built-in datasets
- Learn about graph neural networks for molecules
- Study transfer learning in drug discovery
- Investigate uncertainty quantification in molecular predictions